# YOLOモデル学習（S3ベース）

このノートブックでは、`preprocess_raw_to_yolo_and_pack.ipynb`で作成してS3にアップロードしたYOLOデータセットを使用して、**YOLOv5nモデルを学習**し、学習済み重みをS3に保存します。

**重要**: このノートブックは**YOLOv5の公式リポジトリ（ultralytics/yolov5）**を使用して学習を行います。`ultralytics`パッケージ（YOLOv8）ではなく、確実にYOLOv5で学習できます。

## 処理の流れ
1. AWS認証設定
2. YOLOv5リポジトリのクローンとセットアップ
3. S3からデータセットをダウンロード・展開
4. データセットの確認
5. data.yamlの作成
6. 学習設定
7. YOLOv5のtrain.pyで学習の実行
8. 学習結果の確認
9. モデル重みをS3にアップロード

## 前提条件
- `preprocess_raw_to_yolo_and_pack.ipynb`でデータセットを作成し、S3にアップロード済みであること
- GPUランタイムで実行すること（推奨）
- Gitが利用可能であること（YOLOv5リポジトリのクローンに必要）


## セル0: AWS認証設定


In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = "AKIAxxxxxxxxxxxxxxxx"  # 埋めてね！！！！！！！
os.environ["AWS_SECRET_ACCESS_KEY"] = "xxxxxxxxxxxxxxxxxxxx"  # 埋めてね！！！！！！！
os.environ["AWS_DEFAULT_REGION"] = "ap-northeast-1"

print("AWS env set")

assert "AWS_ACCESS_KEY_ID" in os.environ, "AWS_ACCESS_KEY_ID not set"
assert "AWS_SECRET_ACCESS_KEY" in os.environ, "AWS_SECRET_ACCESS_KEY not set"


## セル1: 設定（ここだけ触ればOK）


In [ ]:
# ============================================
# 設定（ここだけ変更すればOK）
# ============================================

import os
from pathlib import Path

# ===== S3設定 =====
S3_BUCKET = "km62m-ml-storage"  # S3バケット名
S3_PREFIX = "yolo-dataset/v1"  # S3プレフィックス（preprocess_raw_to_yolo_and_pack.ipynbで使用したものと同じ）

# ===== ローカルパス設定 =====
LOCAL_DATA_DIR = Path("/content/tmp/yolo_dataset")  # データセットの展開先
LOCAL_OUTPUT_DIR = Path("/content/tmp/yolo_training")  # 学習結果の出力先

# ===== 学習設定 =====
EPOCHS = 50  # エポック数
BATCH_SIZE = 16  # バッチサイズ（GPUメモリに応じて調整）
IMG_SIZE = 640  # 画像サイズ
DEVICE = 0  # GPU使用（0: GPU, 'cpu': CPU）

# ===== モデル設定 =====
MODEL_NAME = "yolov5n"  # YOLOv5nモデルを使用
PRETRAINED_WEIGHTS = "yolov5n.pt"  # 事前学習済み重み

# ===== その他 =====
CLEAN_LOCAL_DATA = False  # Trueにすると既存のローカルデータを削除

print("✓ 設定完了")
print(f"  S3_BUCKET: {S3_BUCKET}")
print(f"  S3_PREFIX: {S3_PREFIX}")
print(f"  LOCAL_DATA_DIR: {LOCAL_DATA_DIR}")
print(f"  EPOCHS: {EPOCHS}")
print(f"  BATCH_SIZE: {BATCH_SIZE}")
print(f"  IMG_SIZE: {IMG_SIZE}")


## セル2: 依存パッケージのインストール


In [ ]:
# ============================================
# YOLOv5リポジトリのクローンと依存パッケージのインストール
# ============================================

import subprocess
import sys
from pathlib import Path

print("=" * 80)
print("YOLOv5リポジトリのセットアップ")
print("=" * 80)

# YOLOv5リポジトリのパス
YOLOV5_DIR = Path("/content/yolov5")

# YOLOv5リポジトリをクローン（既に存在する場合はスキップ）
if YOLOV5_DIR.exists() and (YOLOV5_DIR / "train.py").exists():
    print(f"\n✓ YOLOv5リポジトリは既にクローン済み: {YOLOV5_DIR}")
else:
    print(f"\n📦 YOLOv5リポジトリをクローン中...")
    print("   URL: https://github.com/ultralytics/yolov5")
    
    # 既存のディレクトリがあれば削除
    if YOLOV5_DIR.exists():
        import shutil
        shutil.rmtree(YOLOV5_DIR)
    
    subprocess.check_call([
        "git", "clone", 
        "https://github.com/ultralytics/yolov5.git",
        str(YOLOV5_DIR)
    ])
    print(f"✓ YOLOv5リポジトリのクローン完了")

# YOLOv5の依存パッケージをインストール
print(f"\n📦 YOLOv5の依存パッケージをインストール中...")
requirements_file = YOLOV5_DIR / "requirements.txt"
if requirements_file.exists():
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", "-q", 
        "-r", str(requirements_file)
    ])
    print(f"✓ YOLOv5の依存パッケージのインストール完了")
else:
    print(f"⚠️  警告: requirements.txtが見つかりません")

# AWS CLI（S3アクセス用）
try:
    result = subprocess.run(["aws", "--version"], capture_output=True, text=True)
    print(f"\n✓ AWS CLI は既にインストール済み: {result.stdout.strip()}")
except FileNotFoundError:
    print("\n📦 AWS CLI をインストール中...")
    subprocess.check_call(["pip", "install", "-q", "awscli"])
    print("✓ AWS CLI インストール完了")

print(f"\n✓ YOLOv5のセットアップ完了")
print(f"  YOLOv5ディレクトリ: {YOLOV5_DIR}")
print(f"  train.py: {YOLOV5_DIR / 'train.py'}")


## セル3: S3からデータセットをダウンロード・展開


In [ ]:
# ============================================
# S3からデータセットをダウンロード・展開
# ============================================

import subprocess
import tarfile
import shutil
from pathlib import Path
from tqdm import tqdm

print("=" * 80)
print("S3からデータセットをダウンロード・展開")
print("=" * 80)

# 既存データのクリーンアップ
if CLEAN_LOCAL_DATA and LOCAL_DATA_DIR.exists():
    print(f"🗑️  既存のローカルデータを削除: {LOCAL_DATA_DIR}")
    shutil.rmtree(LOCAL_DATA_DIR)

# ダウンロード先ディレクトリ
download_dir = LOCAL_DATA_DIR.parent / "yolo_tars_download"
download_dir.mkdir(parents=True, exist_ok=True)

# ダウンロードするtarファイルのリスト
tar_files = [
    "train_images.tar",
    "train_labels.tar",
    "val_images.tar",
    "val_labels.tar",
    "meta.tar"
]

# S3からダウンロード
print(f"\n📥 S3からtarファイルをダウンロード中...")
print(f"   S3パス: s3://{S3_BUCKET}/{S3_PREFIX}/")

downloaded_files = []
for tar_name in tar_files:
    s3_path = f"s3://{S3_BUCKET}/{S3_PREFIX}/{tar_name}"
    local_tar_path = download_dir / tar_name
    
    if local_tar_path.exists():
        size_mb = local_tar_path.stat().st_size / (1024 * 1024)
        print(f"\n✓ {tar_name} は既にダウンロード済み: {size_mb:.2f} MB")
        downloaded_files.append(local_tar_path)
        continue
    
    print(f"\n📥 {tar_name} をダウンロード中...")
    try:
        result = subprocess.run(
            ["aws", "s3", "cp", s3_path, str(local_tar_path)],
            capture_output=True, text=True, check=True
        )
        size_mb = local_tar_path.stat().st_size / (1024 * 1024)
        print(f"  ✓ 完了: {size_mb:.2f} MB")
        downloaded_files.append(local_tar_path)
    except subprocess.CalledProcessError as e:
        print(f"  ❌ 失敗: {e.stderr}")
        raise

# データセットディレクトリの準備
LOCAL_DATA_DIR.mkdir(parents=True, exist_ok=True)
train_images_dir = LOCAL_DATA_DIR / "train" / "images"
train_labels_dir = LOCAL_DATA_DIR / "train" / "labels"
val_images_dir = LOCAL_DATA_DIR / "val" / "images"
val_labels_dir = LOCAL_DATA_DIR / "val" / "labels"

# tarファイルを展開
print(f"\n📦 tarファイルを展開中...")

# train_images.tar
train_images_tar = download_dir / "train_images.tar"
if train_images_tar.exists():
    if not train_images_dir.exists() or len(list(train_images_dir.glob("*"))) == 0:
        train_images_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n📦 train_images.tar を展開中...")
        with tarfile.open(train_images_tar, 'r') as tar:
            tar.extractall(train_images_dir)
        print(f"  ✓ 完了: {len(list(train_images_dir.glob('*')))} ファイル")
    else:
        print(f"\n✓ train_images は既に展開済み: {len(list(train_images_dir.glob('*')))} ファイル")

# train_labels.tar
train_labels_tar = download_dir / "train_labels.tar"
if train_labels_tar.exists():
    if not train_labels_dir.exists() or len(list(train_labels_dir.glob("*.txt"))) == 0:
        train_labels_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n📦 train_labels.tar を展開中...")
        with tarfile.open(train_labels_tar, 'r') as tar:
            tar.extractall(train_labels_dir)
        print(f"  ✓ 完了: {len(list(train_labels_dir.glob('*.txt')))} ファイル")
    else:
        print(f"\n✓ train_labels は既に展開済み: {len(list(train_labels_dir.glob('*.txt')))} ファイル")

# val_images.tar
val_images_tar = download_dir / "val_images.tar"
if val_images_tar.exists():
    if not val_images_dir.exists() or len(list(val_images_dir.glob("*"))) == 0:
        val_images_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n📦 val_images.tar を展開中...")
        with tarfile.open(val_images_tar, 'r') as tar:
            tar.extractall(val_images_dir)
        print(f"  ✓ 完了: {len(list(val_images_dir.glob('*')))} ファイル")
    else:
        print(f"\n✓ val_images は既に展開済み: {len(list(val_images_dir.glob('*')))} ファイル")

# val_labels.tar
val_labels_tar = download_dir / "val_labels.tar"
if val_labels_tar.exists():
    if not val_labels_dir.exists() or len(list(val_labels_dir.glob("*.txt"))) == 0:
        val_labels_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n📦 val_labels.tar を展開中...")
        with tarfile.open(val_labels_tar, 'r') as tar:
            tar.extractall(val_labels_dir)
        print(f"  ✓ 完了: {len(list(val_labels_dir.glob('*.txt')))} ファイル")
    else:
        print(f"\n✓ val_labels は既に展開済み: {len(list(val_labels_dir.glob('*.txt')))} ファイル")

# meta.tar（mapping.json, data.yaml, manifest.json）
meta_tar = download_dir / "meta.tar"
if meta_tar.exists():
    print(f"\n📦 meta.tar を展開中...")
    with tarfile.open(meta_tar, 'r') as tar:
        tar.extractall(LOCAL_DATA_DIR)
    print(f"  ✓ 完了")

# manifest.jsonとdata.yamlの確認
manifest_file = LOCAL_DATA_DIR / "manifest.json"
data_yaml_file = LOCAL_DATA_DIR / "data.yaml"

if manifest_file.exists():
    print(f"\n✓ manifest.json を確認: {manifest_file}")
if data_yaml_file.exists():
    print(f"✓ data.yaml を確認: {data_yaml_file}")
else:
    print(f"\n⚠️  警告: data.yaml が見つかりません。後で作成します。")

print(f"\n✓ データセットのダウンロード・展開完了")
print(f"  データセットパス: {LOCAL_DATA_DIR}")


## セル4: データセットの確認


In [ ]:
# ============================================
# データセットの確認
# ============================================

import json
from collections import defaultdict

print("=" * 80)
print("データセット確認")
print("=" * 80)

# ファイル数の確認
train_images = list(train_images_dir.glob("*"))
train_labels = list(train_labels_dir.glob("*.txt"))
val_images = list(val_images_dir.glob("*"))
val_labels = list(val_labels_dir.glob("*.txt"))

print(f"\nTrainデータ:")
print(f"  画像: {len(train_images)} 件")
print(f"  ラベル: {len(train_labels)} 件")

print(f"\nValデータ:")
print(f"  画像: {len(val_images)} 件")
print(f"  ラベル: {len(val_labels)} 件")

# 1対1対応の確認
train_image_uuids = {f.stem for f in train_images}
train_label_uuids = {f.stem for f in train_labels}
val_image_uuids = {f.stem for f in val_images}
val_label_uuids = {f.stem for f in val_labels}

train_matched = train_image_uuids & train_label_uuids
val_matched = val_image_uuids & val_label_uuids

print(f"\n1対1対応確認:")
print(f"  Train: {len(train_matched)}/{len(train_images)} ペア")
print(f"  Val: {len(val_matched)}/{len(val_images)} ペア")

# manifest.jsonの読み込み（クラス情報を取得）
if manifest_file.exists():
    with open(manifest_file, 'r', encoding='utf-8') as f:
        manifest = json.load(f)
    
    target_classes = manifest.get('target_classes', [])
    print(f"\nmanifest.jsonから取得した情報:")
    print(f"  対象クラス数: {len(target_classes)}")
    print(f"  クラス: {', '.join(target_classes)}")
    print(f"  Train数: {manifest.get('split', {}).get('train_count', 'N/A')}")
    print(f"  Val数: {manifest.get('split', {}).get('val_count', 'N/A')}")
else:
    print(f"\n⚠️  警告: manifest.json が見つかりません")
    target_classes = None

print(f"\n✓ データセット確認完了")


## セル5: data.yamlの作成（manifest.jsonから）


In [ ]:
# ============================================
# data.yamlの作成（manifest.jsonから）
# ============================================

import yaml

print("=" * 80)
print("data.yaml作成")
print("=" * 80)

# manifest.jsonから情報を取得
if not manifest_file.exists():
    raise FileNotFoundError(f"manifest.jsonが見つかりません: {manifest_file}")

with open(manifest_file, 'r', encoding='utf-8') as f:
    manifest = json.load(f)

target_classes = manifest.get('target_classes', [])
if not target_classes:
    raise ValueError("manifest.jsonにtarget_classesが含まれていません")

# data.yamlを作成
data_config = {
    'path': str(LOCAL_DATA_DIR.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(target_classes),
    'names': {idx: cls for idx, cls in enumerate(target_classes)}
}

# 既存のdata.yamlがある場合は上書き
with open(data_yaml_file, 'w', encoding='utf-8') as f:
    yaml.dump(data_config, f, default_flow_style=False, allow_unicode=True)

print(f"\n✓ data.yamlを作成: {data_yaml_file}")
print(f"\n内容:")
print(yaml.dump(data_config, default_flow_style=False, allow_unicode=True))

data_path = str(data_yaml_file)


## セル6: 事前学習済みモデルの準備


In [ ]:
# ============================================
# YOLOv5の事前学習済み重みの確認
# ============================================

import sys
from pathlib import Path

print("=" * 80)
print("YOLOv5の事前学習済み重みの確認")
print("=" * 80)

# YOLOv5のtrain.pyは初回実行時に自動的に重みをダウンロードします
# ここでは設定を確認するだけです

print(f"\n✓ YOLOv5nモデルを使用します")
print(f"  モデル名: {MODEL_NAME}")
print(f"  事前学習済み重み: {PRETRAINED_WEIGHTS}")
print(f"  （train.py実行時に自動ダウンロードされます）")
print(f"\n✓ データセット情報:")
print(f"  クラス数: {len(target_classes)} クラス")
print(f"  クラス: {', '.join(target_classes)}")
print(f"  data.yaml: {data_path}")


## セル7: 学習設定


In [ ]:
# ============================================
# 学習設定
# ============================================

from datetime import datetime

print("=" * 80)
print("学習設定")
print("=" * 80)

# 出力先の設定
LOCAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 実行名（タイムスタンプ付き）
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"{MODEL_NAME}_train_{timestamp}"

print(f"\nデータセット: {data_path}")
print(f"エポック数: {EPOCHS}")
print(f"バッチサイズ: {BATCH_SIZE}")
print(f"画像サイズ: {IMG_SIZE}")
print(f"デバイス: {'GPU' if DEVICE == 0 else 'CPU'}")
print(f"出力先: {LOCAL_OUTPUT_DIR}/{run_name}")
print(f"\nクラス数: {len(target_classes)} クラス")
print(f"クラス: {', '.join(target_classes)}")


## セル8: 学習の実行

**注意**: このセルは長時間かかります（数時間）。GPUランタイムを使用してください。


In [ ]:
# ============================================
# 学習の実行（YOLOv5のtrain.pyを使用）
# ============================================

import subprocess
import sys
from pathlib import Path

print("=" * 80)
print("学習開始（YOLOv5 train.py）")
print("=" * 80)
print(f"開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# YOLOv5のtrain.pyのパス
train_script = YOLOV5_DIR / "train.py"

if not train_script.exists():
    raise FileNotFoundError(f"train.pyが見つかりません: {train_script}")

# train.pyの実行コマンドを構築
train_cmd = [
    sys.executable,
    str(train_script),
    "--data", str(data_path),
    "--weights", PRETRAINED_WEIGHTS,  # yolov5n.pt（自動ダウンロード）
    "--epochs", str(EPOCHS),
    "--batch-size", str(BATCH_SIZE),
    "--img", str(IMG_SIZE),
    "--device", str(DEVICE),
    "--project", str(LOCAL_OUTPUT_DIR),
    "--name", run_name,
    "--save-period", "10",  # 10エポックごとに保存
    "--val",  # 検証データで評価
    "--plots",  # 学習曲線をプロット
]

print(f"実行コマンド:")
print(f"  {' '.join(train_cmd)}")
print()

# 作業ディレクトリをYOLOv5に変更して実行
# （YOLOv5のtrain.pyは相対パスでリソースを参照するため）
original_cwd = Path.cwd()
try:
    os.chdir(YOLOV5_DIR)
    print(f"作業ディレクトリ: {YOLOV5_DIR}")
    print()
    
    # 学習実行
    result = subprocess.run(
        train_cmd,
        check=True,
        text=True
    )
    
    print("\n" + "=" * 80)
    print("学習完了")
    print("=" * 80)
    print(f"終了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
except subprocess.CalledProcessError as e:
    print(f"\n❌ エラー: 学習中にエラーが発生しました")
    print(f"   終了コード: {e.returncode}")
    raise
finally:
    os.chdir(original_cwd)


## セル9: 学習結果の確認


In [ ]:
# ============================================
# 学習結果の確認
# ============================================

results_dir = LOCAL_OUTPUT_DIR / run_name
weights_dir = results_dir / "weights"

print("=" * 80)
print("学習結果")
print("=" * 80)
print(f"結果ディレクトリ: {results_dir}")

# 重みファイルの確認
if weights_dir.exists():
    weight_files = list(weights_dir.glob('*.pt'))
    print(f"\n重みファイル数: {len(weight_files)} 件")
    
    for wf in sorted(weight_files):
        size_mb = wf.stat().st_size / (1024 * 1024)
        print(f"  {wf.name}: {size_mb:.2f} MB")
    
    # 最良モデル
    best_model = weights_dir / "best.pt"
    if best_model.exists():
        print(f"\n✓ 最良モデル: {best_model.name}")
        print(f"  サイズ: {best_model.stat().st_size / (1024 * 1024):.2f} MB")
    
    # 最終モデル
    last_model = weights_dir / "last.pt"
    if last_model.exists():
        print(f"✓ 最終モデル: {last_model.name}")
        print(f"  サイズ: {last_model.stat().st_size / (1024 * 1024):.2f} MB")
else:
    print("\n⚠️  重みファイルが見つかりません")

# 学習曲線の確認
results_csv = results_dir / "results.csv"
if results_csv.exists():
    print(f"\n学習曲線データ: {results_csv}")

# プロット画像の確認
plot_files = list(results_dir.glob("*.png"))
if len(plot_files) > 0:
    print(f"\nプロット画像数: {len(plot_files)} 件")
    for pf in plot_files:
        print(f"  {pf.name}")


## セル10: モデル重みをS3にアップロード


In [ ]:
# ============================================
# モデル重みをS3にアップロード
# ============================================

import subprocess
import time

print("=" * 80)
print("モデル重みをS3にアップロード")
print("=" * 80)

# AWS認証情報の確認
try:
    result = subprocess.run(["aws", "sts", "get-caller-identity"], 
                          capture_output=True, text=True, check=True)
    print(f"\n✓ AWS認証情報を確認")
except subprocess.CalledProcessError:
    print("\n❌ エラー: AWS認証情報が設定されていません")
    print("   セル0でAWS認証情報を設定してください")
    raise

# S3の保存先パス
s3_weights_prefix = f"{S3_PREFIX}/weights/{run_name}"

# アップロードするファイルのリスト
files_to_upload = []

# 1. 最良モデル（best.pt）
best_model = weights_dir / "best.pt"
if best_model.exists():
    files_to_upload.append(("最良モデル", best_model, f"{s3_weights_prefix}/best.pt"))
else:
    print("⚠️  最良モデル（best.pt）が見つかりません")

# 2. 最終モデル（last.pt）
last_model = weights_dir / "last.pt"
if last_model.exists():
    files_to_upload.append(("最終モデル", last_model, f"{s3_weights_prefix}/last.pt"))

# 3. 学習曲線データ（results.csv）
results_csv = results_dir / "results.csv"
if results_csv.exists():
    files_to_upload.append(("学習曲線データ", results_csv, f"{s3_weights_prefix}/results.csv"))

# 4. data.yaml（学習に使用した設定）
if data_yaml_file.exists():
    files_to_upload.append(("data.yaml", data_yaml_file, f"{s3_weights_prefix}/data.yaml"))

# 5. manifest.json（データセット情報）
if manifest_file.exists():
    files_to_upload.append(("manifest.json", manifest_file, f"{s3_weights_prefix}/manifest.json"))

print(f"\nアップロードするファイル数: {len(files_to_upload)} 件")
print(f"S3パス: s3://{S3_BUCKET}/{s3_weights_prefix}/")
print()

# ファイルをアップロード
uploaded = []
failed = []

for desc, local_file, s3_key in files_to_upload:
    if not local_file.exists():
        print(f"⚠️  スキップ: {desc}（ファイルが見つかりません）")
        continue
    
    s3_path = f"s3://{S3_BUCKET}/{s3_key}"
    size_mb = local_file.stat().st_size / (1024 * 1024)
    
    print(f"\n📤 {desc} をアップロード中...")
    print(f"   ファイル: {local_file.name}")
    print(f"   サイズ: {size_mb:.2f} MB")
    print(f"   S3パス: {s3_path}")
    
    max_retries = 3
    retry_count = 0
    success = False
    
    while retry_count < max_retries and not success:
        try:
            start_time = time.time()
            result = subprocess.run(
                ["aws", "s3", "cp", str(local_file), s3_path],
                capture_output=True, text=True, check=True
            )
            elapsed = time.time() - start_time
            print(f"  ✓ 完了: {elapsed:.1f}秒")
            uploaded.append(desc)
            success = True
        except subprocess.CalledProcessError as e:
            retry_count += 1
            if retry_count < max_retries:
                print(f"  ⚠️  リトライ {retry_count}/{max_retries}...")
                time.sleep(2)
            else:
                print(f"  ❌ 失敗: {e.stderr}")
                failed.append(desc)

print(f"\n" + "=" * 80)
print("アップロード結果")
print("=" * 80)
print(f"  成功: {len(uploaded)} 件")
if uploaded:
    for name in uploaded:
        print(f"    ✓ {name}")

print(f"  失敗: {len(failed)} 件")
if failed:
    for name in failed:
        print(f"    ❌ {name}")

if len(uploaded) == len(files_to_upload):
    print(f"\n✓ 全てのファイルのアップロードが完了しました")
    print(f"  S3パス: s3://{S3_BUCKET}/{s3_weights_prefix}/")
    print(f"\n次回は以下のパスからモデルを読み込めます:")
    print(f"  最良モデル: s3://{S3_BUCKET}/{s3_weights_prefix}/best.pt")
else:
    print(f"\n⚠️  一部のファイルのアップロードに失敗しました")
    print(f"  失敗したファイルは手動でアップロードしてください")
